In [1]:
#import git
#from git.repo.base import Repo
#Repo.clone_from("https://github.com/PhonePe/pulse.git", "./Data")

In [38]:
import pandas as pd
import numpy as np
import os
import json
import mysql.connector

In [42]:
data_base_path = "./Data/data/aggregated/transaction/country/india/state/"
#Aggregated data
aggr_transactions_list = os.listdir(data_base_path)


In [44]:
#iterating states
mydata = {}

master_set = {'State': [], 'Year': [], 'Quarter': [], 'Transaction Type': [], 'Transactions count': [],
            'Transaction Amount': []}

for state in aggr_transactions_list:
    cur_state = data_base_path + state + "/"
    years_list = os.listdir(cur_state)
    #iterating years for a state
    for year in years_list:
        cur_year = cur_state + year + "/"
        files_list = os.listdir(cur_year)
        for file in files_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            mydata = json.load(data)
            txns = mydata['data']['transactionData']
            for txn in txns:
                name = txn['name']
                counts = txn['paymentInstruments'][0]['count']
                txnamount = txn['paymentInstruments'][0]['amount']
                master_set['Transaction Type'].append(name)
                master_set['Transactions count'].append(counts)
                master_set['Transaction Amount'].append(txnamount)
                master_set['State'].append(state)
                master_set['Year'].append(year)
                master_set['Quarter'].append(int(file.strip('.json')))

df_aggregation_txns = pd.DataFrame(master_set)


In [5]:
user_data_path = "./Data/data/aggregated/user/country/india/state/"
users_aggrefation_list = os.listdir(user_data_path)


In [6]:
myuserdata = {}
utxns = []
user_master_set = {'State': [], 'Year': [], 'Quarter': [], 'Brands': [], 'Count': [],'Percentage': []}

for state in users_aggrefation_list:
    cur_state = user_data_path + state + "/"
    years_list = os.listdir(cur_state)
    #iterating years for a state
    for year in years_list:
        cur_year = cur_state + year + "/"
        files_list = os.listdir(cur_year)
        for file in files_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            myuserdata = json.load(data)
            #print(state,year)
            utxns = myuserdata['data']['usersByDevice']
            #print(utxns)
            if utxns != None:
                for txn in utxns:
                    brand = txn['brand']
                    counts = txn['count']
                    percent = txn['percentage']
                    user_master_set['Brands'].append(brand)
                    user_master_set['Year'].append(year)
                    user_master_set['Count'].append(counts)
                    user_master_set['State'].append(state)
                    user_master_set['Quarter'].append(int(file.strip('.json')))
                    user_master_set['Percentage'].append(percent)
            
df_users_txn = pd.DataFrame(user_master_set)
#df_users_txn


,State,Year,Quarter,Brands,Count,Percentage
0,andaman-&-nicobar-islands,2018,1,Xiaomi,1665,0.247033
1,andaman-&-nicobar-islands,2018,1,Samsung,1445,0.214392
2,andaman-&-nicobar-islands,2018,1,Vivo,982,0.145697
3,andaman-&-nicobar-islands,2018,1,Oppo,501,0.074332
4,andaman-&-nicobar-islands,2018,1,OnePlus,332,0.049258
...,...,...,...,...,...,...
6727,west-bengal,2022,1,Lenovo,330017,0.015056
6728,west-bengal,2022,1,Infinix,284678,0.012987
6729,west-bengal,2022,1,Asus,280347,0.012790
6730,west-bengal,2022,1,Apple,277752,0.012671


In [7]:
#getting map data
map_data_path = "./Data/data/map/transaction/hover/country/india/state/"
map_txn_list = os.listdir(map_data_path)

In [8]:
map_txn_data = {}
map_txn_set = {'State': [], 'Year': [], 'Quarter': [], 'District': [], 'Count': [],
            'Amount': []}
for state in map_txn_list:
    cur_state = map_data_path + state + "/"
    map_years = os.listdir(cur_state)
    for year in map_years:
        cur_year = cur_state + year + "/"
        map_file_list = os.listdir(cur_year)
        for file in map_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            map_txn_data = json.load(data)

            for tnx in map_txn_data["data"]["hoverDataList"]:
                district = tnx["name"]
                count = tnx["metric"][0]["count"]
                amount = tnx["metric"][0]["amount"]
                map_txn_set["District"].append(district)
                map_txn_set["Count"].append(count)
                map_txn_set["Amount"].append(amount)
                map_txn_set['State'].append(state)
                map_txn_set['Year'].append(year)
                map_txn_set['Quarter'].append(int(file.strip('.json')))
            
df_map_txn = pd.DataFrame(map_txn_set)


,State,Year,Quarter,District,Count,Amount
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,442,9.316631e+05
1,andaman-&-nicobar-islands,2018,1,south andaman district,5688,1.256025e+07
2,andaman-&-nicobar-islands,2018,1,nicobars district,528,1.139849e+06
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,825,1.317863e+06
4,andaman-&-nicobar-islands,2018,2,south andaman district,9395,2.394824e+07
...,...,...,...,...,...,...
16095,west-bengal,2023,2,nadia district,26194658,5.410386e+10
16096,west-bengal,2023,2,birbhum district,14912523,2.941312e+10
16097,west-bengal,2023,2,purba medinipur district,30724502,6.595899e+10
16098,west-bengal,2023,2,maldah district,24677357,5.218621e+10


In [9]:
map_user_path = "./Data/data/map/user/hover/country/india/state/"
map_user_txn_list = os.listdir(map_user_path)

In [10]:
mapuserdata={}
map_user_set = {"State": [], "Year": [], "Quarter": [], "District": [],
            "RegisteredUser": [], "AppOpens": []}
for state in map_user_txn_list:
    cstate = map_user_path+state+"/"
    yearslist = os.listdir(cstate)
    for year in yearslist:
        cur_year = cstate + year + "/"
        map_file_list = os.listdir(cur_year)
        for file in map_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            mapuserdata = json.load(data)
            for user in mapuserdata["data"]["hoverData"].items():
                district = user[0]
                registereduser = user[1]["registeredUsers"]
                appOpens = user[1]['appOpens']
                map_user_set["District"].append(district)
                map_user_set["RegisteredUser"].append(registereduser)
                map_user_set["AppOpens"].append(appOpens)
                map_user_set['State'].append(state)
                map_user_set['Year'].append(year)
                map_user_set['Quarter'].append(int(file.strip('.json')))

df_map_user = pd.DataFrame(map_user_set)
                

,State,Year,Quarter,District,RegisteredUser,AppOpens
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,632,0
1,andaman-&-nicobar-islands,2018,1,south andaman district,5846,0
2,andaman-&-nicobar-islands,2018,1,nicobars district,262,0
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,911,0
4,andaman-&-nicobar-islands,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
16099,west-bengal,2023,2,nadia district,1482400,9509833
16100,west-bengal,2023,2,birbhum district,934943,12789154
16101,west-bengal,2023,2,purba medinipur district,1468608,12096564
16102,west-bengal,2023,2,maldah district,1046132,17662446


In [20]:
top_trans_path = "./Data/data/top/transaction/country/india/state/"
top_txn_list = os.listdir(top_trans_path)

In [28]:
top_trans_data = {}
fileslist=[]
top_transaction = os.listdir(top_trans_path)
top_transaction_set = {'State': [], 'Year': [], 'Quarter': [], 'District': [], 'Transaction_count': [],
            'Amount': []}
for state in top_transaction:
    cstate = top_trans_path + state+"/"
    #print(cstate)
    topyearslists = os.listdir(cstate)
    for year in topyearslists:
        cyear = cstate+year +"/"
        #print(cyear)
        topfiles = os.listdir(cyear)
        for file in topfiles:
            cfile = cyear + file
            data = open(cfile,'r')
            top_trans_data = json.load(data)
            for dist in top_trans_data['data']['districts']:
                name=dist['entityName']
                counts=dist['metric']['count']
                amt = dist['metric']['amount']
                top_transaction_set['Amount'].append(amt)
                top_transaction_set['Year'].append(year)
                top_transaction_set['State'].append(state)
                top_transaction_set['District'].append(name)
                top_transaction_set['Quarter'].append(int(file.strip('.json')))
                top_transaction_set['Transaction_count'].append(counts)
df_top_trans = pd.DataFrame(top_transaction_set)      

                
            
#top_trans_data
        #topfiles = os.listdir(cyear)
#topfiles 
        
#df_top_trans = pd.DataFrame(top_transaction_set)
#df_top_trans

,State,Year,Quarter,District,Transaction_count,Amount
0,andaman-&-nicobar-islands,2018,1,south andaman,5688,1.256025e+07
1,andaman-&-nicobar-islands,2018,1,nicobars,528,1.139849e+06
2,andaman-&-nicobar-islands,2018,1,north and middle andaman,442,9.316631e+05
3,andaman-&-nicobar-islands,2018,2,south andaman,9395,2.394824e+07
4,andaman-&-nicobar-islands,2018,2,nicobars,1120,3.072437e+06
...,...,...,...,...,...,...
6507,west-bengal,2023,2,hooghly,27753013,5.209866e+10
6508,west-bengal,2023,2,howrah,27156689,5.352124e+10
6509,west-bengal,2023,2,paschim medinipur,26646934,4.907191e+10
6510,west-bengal,2023,2,nadia,26194658,5.410386e+10


In [31]:
top_user_paths = "./Data/data/top/user/country/india/state/"
top_user_list = os.listdir(top_user_paths)
#top_user_list

In [56]:
top_users_set = {'State': [], 'Year': [], 'Quarter': [], 'District': [],
            'RegisteredUsers': []}
for state in top_user_list:
    cstate = top_user_paths + state +"/"
    years = os.listdir(cstate)
    
    for year in years:
        cyear = cstate + year+'/'
        files = os.listdir(cyear)

        for file in files:
            cfile = cyear + file
            #print(cfile)
            content = open(cfile,'r')
            data = json.load(content)

            for dist in data['data']['districts']:
                name = dist['name']
                register_users = dist['registeredUsers']
                top_users_set['Year'].append(year)
                top_users_set['District'].append(name)
                top_users_set['Quarter'].append(int(file.strip('.json')))
                top_users_set['RegisteredUsers'].append(register_users)
                top_users_set['State'].append(state)
                
                
df_top_users = pd.DataFrame(top_users_set)          
df_top_users

,State,Year,Quarter,District,RegisteredUsers
0,andaman-&-nicobar-islands,2018,1,south andaman,5846
1,andaman-&-nicobar-islands,2018,1,north and middle andaman,632
2,andaman-&-nicobar-islands,2018,1,nicobars,262
3,andaman-&-nicobar-islands,2018,2,south andaman,8143
4,andaman-&-nicobar-islands,2018,2,north and middle andaman,911
...,...,...,...,...,...
6507,west-bengal,2023,2,howrah,1546777
6508,west-bengal,2023,2,nadia,1482400
6509,west-bengal,2023,2,purba medinipur,1468608
6510,west-bengal,2023,2,paschim medinipur,1328693


In [45]:
df_aggregation_txns.to_csv('aggregation_txns.csv',index=False)
df_users_txn.to_csv('users_txn.csv',index=False)
df_map_txn.to_csv('map_trans.csv',index=False)
df_map_user.to_csv('map_user.csv',index=False)
df_top_trans.to_csv('top_trans.csv',index=False)
df_top_users.to_csv('top_user.csv',index=False)

In [41]:
#MySQL Integration 
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",database='research'
    )
cursor = mydb.cursor()


In [47]:
# inserting aggregated transactions....
query = "insert into PHONEPE_AGGREGATED_TRANS values(%s,%s,%s,%s,%s,%s)"
records = []
for i,row in df_aggregation_txns.iterrows():
    records.append(tuple(row))
cursor.executemany(query,records)
print('records inserted successfully')


records inserted successfully


In [48]:
mydb.commit()

In [49]:
query = "insert into PHONEPE_AGGREGATED_USERS values(%s,%s,%s,%s,%s,%s)"
records = []
for i,row in df_users_txn.iterrows():
    records.append(tuple(row))
cursor.executemany(query,records)
print('records inserted successfully')
mydb.commit()

records inserted successfully


In [50]:
#Map transactions...
query = "insert into PHONEPE_MAP_TRANSACTIONS values(%s,%s,%s,%s,%s,%s)"
records = []
for i,row in df_map_txn.iterrows():
    records.append(tuple(row))
cursor.executemany(query,records)
print('records inserted successfully')
mydb.commit()

records inserted successfully


In [51]:
#Map transactions...
query = "insert into PHONEPE_MAP_USERS values(%s,%s,%s,%s,%s,%s)"
records = []
for i,row in df_map_user.iterrows():
    records.append(tuple(row))
cursor.executemany(query,records)
print('records inserted successfully')
mydb.commit()

records inserted successfully


In [52]:
#Top transactions...
query = "insert into TOP_TRANSACTIONS values(%s,%s,%s,%s,%s,%s)"
records = []
for i,row in df_top_trans.iterrows():
    records.append(tuple(row))
cursor.executemany(query,records)
print('records inserted successfully')
mydb.commit()

records inserted successfully


In [57]:
#Top transactions...
query = "insert into TOP_USERS values(%s,%s,%s,%s,%s)"
records = []
for i,row in df_top_users.iterrows():
    records.append(tuple(row))
cursor.executemany(query,records)
print('records inserted successfully')
mydb.commit()

records inserted successfully


In [58]:
cursor.close()

True